# dev - test driven elastic forces on a known deformation on a sphere 
Tim Tyree<br>


# TODO: pen & paper out easily njit'd compute_elastic_force implementation
- TODO: develop ^that function here, testing as I go. (proceed to next section).

## import a sphere and initialize environment

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
# sys.path.append("../lib") 
# from lib import *
# sys.path.append("lib") 
from lib import *

# from operari import *
# from ProgressBar import *
# from mesh_ops import *

# the visualization tools involved here for triangular meshes is
import trimesh
import pyglet
from numba import njit, cuda
# from numba.typed import List
# import numba
import trimesh

#try using a scipy sparse matrix to speed up spring force evaluations
#TODO: consider speeding up bigger meshes with pycuda's sparce matrices
from scipy.sparse import csr_matrix
import scipy.sparse as sparse

#formating
from matplotlib import rc
rc('text', usetex=True)

%autocall 1
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'vertex_shader'

In [95]:
os.chdir(f'{nb_dir}/Data/spherical_meshes')
mesh = trimesh.load('spherical_mesh_64.stl')
#subtract the center of mass
mesh.vertices -= mesh.center_mass
#normalize the mean radius to 1
mesh.vertices /= np.cbrt(mesh.volume*3/(4*np.pi))

#normalize the mean radius to R
# R = 1. #unit length
# mesh.vertices *= R/np.mean(np.linalg.norm(mesh.vertices,axis=1))

face_normals all zero, ignoring!


# compute geometric measures of the initial configuration, $\Omega_0$.

In [55]:
def measure(mesh, verbose=False):
    vol = mesh.volume
    #two estimates of radius, which we combine for a radial uncertainty estimate
    radius_vol = np.cbrt(mesh.volume*3/(4*np.pi))
    radius_rms = np.mean(np.linalg.norm(mesh.vertices,axis=1))        
    radius_avg = radius_rms/2 + radius_vol/2
    radius_err = radius_rms/2 - radius_vol/2
    #measure surface area to volume ratio
    svr = mesh.area/vol
    if verbose:
        print(f"vol : {vol}") #root mean squared radius
        print(f"svr : {radius_rms}") #root mean squared radius
        print(f"radius_avg : {radius_avg}") #average radius
        print(f"radius_err : {radius_err}\n") #uncertainty in radius
        print(f"radius_vol : {radius_vol}") #volumetric radius
        print(f"radius_rms : {radius_rms}") #root mean squared radius
    

    return vol, svr, radius_avg, radius_err

In [39]:
#test geoemetric measures
os.chdir(f'{nb_dir}/Data/spherical_meshes')
mesh = trimesh.load('spherical_mesh_64.stl')
#subtract the center of mass
mesh.vertices -= mesh.center_mass
#normalize the mean radius to 1
mesh.vertices /= np.cbrt(mesh.volume*3/(4*np.pi))


Omega0 = mesh.copy()
#test for geometric measure functionality
retval_known = (4.18879020478639,3.0391040405123317,1.0041946729795743,0.0041946729795744875)
retval = measure(Omega0, verbose=True)

chksm = np.array([np.isclose(a,b) for a,b in zip(retval,(4.18879020478639,3.0391040405123317,1.0041946729795743,0.0041946729795744875))])
assert ( chksm.all() ) 

face_normals all zero, ignoring!


vol : 4.18879020478639
svr : 1.0083893459591489
radius_avg : 1.0041946729795743
radius_err : 0.0041946729795744875

radius_vol : 0.9999999999999999
radius_rms : 1.0083893459591489


# contract by a known volume_fraction

In [41]:
volume_fraction = 0.5
Omega0 = mesh.copy()
Omega1 = Omega0.copy()
Omega1.vertices *= volume_fraction

In [54]:
print("\nOmega0 is:{")
retval = measure(Omega0,verbose = True)
print("}\nOmega1 is:{")
retval = measure(Omega1,verbose = True)    
print("}")


Omega0 is:{
vol : 4.18879020478639
svr : 1.0083893459591489
radius_avg : 1.0041946729795743
radius_err : 0.0041946729795744875

radius_vol : 0.9999999999999999
radius_rms : 1.0083893459591489
}
Omega1 is:{
vol : 0.5235987755982987
svr : 0.5041946729795744
radius_avg : 0.5020973364897872
radius_err : 0.0020973364897872437

radius_vol : 0.49999999999999994
radius_rms : 0.5041946729795744
}


# TODO: develop spring force computation, testing as I go. (proceed to next section).


- TODO: the forces for a massless string for a given mesh edge
- TODO: for each edge, compute the spring force vector
- TODO: add the spring forces to get a net spring force for each node.  Consider for loop or a for for loop.
- TODO: use precomputed mesh.edges_unique in computing net_spring_forces
<br><br>
- TODO(viewer): compute the deformation gradient between two input meshes.  see old implementation.  make sure this plotting function is saved
- TODO(viewer): remake that triangle nodal force plot for a spring model.  does it look reasonable?
<br><br>
- TODO(model): init a u = displacement field and ud = velocity field to zero, udd = acceleration field to zero
- TODO(model): compute a drag force for a given vertex at a given speed
- TODO(model - boundary conditions): functionally remove edges that contain vertices in a given set.  this will be precomputed and doesn't need to be fast
<br><br>
- TODO(controller): def integrate_forward_n_steps(mesh, time_step, n_steps): #for time_step from the explicit forward euler method
    - then, TODO: njit ^that
- TODO(controller): make/test the explicit newmark method
- TODO(controller): make/test the implicit newmark method
- TODO(controller, later): use kDTree/BTree _blocked_ data structure to implement a queue at constant time steps. Hint: est = mesh.edges_sorted_tree.
- TODO(controller, later): consider how big of a block is best for using a cpu versus a gpu.  for cpu, it doesn't matter.  for gpu, maybe ~1000 force-generating simplices per block


In [84]:
# mesh.edges_unique.shape?
assert ((mesh.edges_unique[mesh.edges_unique_inverse] == mesh.edges_sorted).all())

def length(vertex1, vertex2):
    return np.linalg.norm( vertex1 - vertex2)

meu = mesh.edges_unique
vertex1_array, vertex2_array = mesh.vertices[meu[:,0]], mesh.vertices[meu[:,1]]

#test mesh.edges_unique_length gives the correct edge length 
assert ( length(vertex1_array[0],vertex2_array[0]) == mesh.edges_unique_length[0] )
assert ( length(vertex1_array[1],vertex2_array[1]) == mesh.edges_unique_length[1] )
assert ( length(vertex1_array[2],vertex2_array[2]) == mesh.edges_unique_length[2] )

#precompute edges, which are the force generating simplices here
meu = mesh.edges_unique
vid1_array, vid2_array = np.array(meu[:,0],dtype=int), np.array( meu[:,1],dtype=int)
vertex1_array = np.array ( mesh.vertices[vid1_array]  , dtype=float) 
vertex2_array = np.array ( mesh.vertices[vid2_array]  , dtype=float) 


#initialize spring force parameters
d0_array = np.array(mesh.edges_unique_length, dtype=float)
k0_array = 1 + np.zeros_like(vertex1_array[...,0], dtype=float)
f0_array = np.array(np.zeros_like(mesh.vertices, dtype=float))


#test type consistency as a proxy test that the types are not tracked-array instances
assert ( type(dist_array) == type (d0_array) )
assert ( type(f0_array) == type (k0_array) )
assert ( type(vertex1_array) == type (d0_array) )
assert ( type(vertex2_array) == type (d0_array) )
assert ( type(vid1_array) == type (d0_array) )
assert ( type(vid2_array) == type (d0_array) )

#TODO(model - boundary conditions): functionally remove edges that contain vertices in a given set.  this will be precomputed and doesn't need to be fast

#compute spring force for a given edge configuration
def spring_force( vertex1, vertex2, d, d0, k0 ):
    '''returns the spring_force for vertex1 directed towards vertex2.  '''
    dhat = (vertex2 - vertex1)
    dhat /= np.linalg.norm(dhat)
    return - k0 * ( d - d0 ) * dhat

# test that spring_force pushes vertex1 in the direction of vertex2 when the spring is twice the equilibrium length, d0
vertex1, vertex2, d, d0, k0  = (np.array([ 0.71367072, -0.45411208,  0.53182509]),np.array([ 0.54048159, -0.63460453,  0.61003465]),0.26208535311672354,0.26208535311672354,1.0)
dhat = (vertex1 - vertex2)
dhat/= np.linalg.norm(dhat)

# test that an extended spring is attractive in the edge direction
f0   = spring_force( vertex1, vertex2, d, d0/2, k0 )
assert ( np.isclose ( f0.dot(dhat)/np.linalg.norm(f0) , 1 ) ) 
# test that a compressed spring is repulsive in the edge direction
f0   = spring_force( vertex1, vertex2, d, d0*2, k0 )
assert ( np.isclose ( f0.dot(dhat)/np.linalg.norm(f0) , -1 ) ) 


#compute the net spring force at each vertex by iterating over the edges modeling springs
dist_array  = np.zeros_like(vertex1_array, dtype=float)
force_array = np.zeros_like(vertex1_array, dtype=float)
vertex1_array, vertex2_array = np.array(mesh.vertices[meu[:,0]],dtype=float), np.array( mesh.vertices[meu[:,1]],dtype=float)
for i, (vertex1, vertex2, d, d0, k0) in enumerate(zip(vertex1_array, vertex2_array, mesh.edges_unique_length, d0_array, k0_array)):
    dist_array[i] = vertex1 - vertex2
    # compute the spring force given edge in the local scope
    f0   = spring_force( vertex1, vertex2, d, d0/2, k0 )

    # add that spring force in the appropriate entries of f0_array
    f0_array[meu[i,0]] += f0
    f0_array[meu[i,1]] -= f0
    
    # TODO(optional, for plotting): compute the average of the vertex vectors

assert (np.isclose( np.mean(f0_array) , 0 ) )  #test average spring force is zero
assert ( np.var(f0_array) > 0.02 )  #test variance of spring force at least 0.02

(756, 2)

# wrap this all in a function compute_spring_forces for a given (mesh, precomputed_values)


In [344]:
# TODO: write a test_compute_spring_forces function
# TODO: njit compute_spring_forces(mesh_world, precomputed_values) function
# TODO: move all this to model.py  and the test_model.py
# TODO(later): njit compute_elastic_forces(mesh_world, mesh_material) function

In [349]:
#compute spring force for a given edge configuration
def spring_force( vertex1, vertex2, d, d0, k0 ):
    '''returns the spring_force for vertex1 directed towards vertex2.  '''
    dhat = (vertex2 - vertex1)
    dhat /= np.linalg.norm(dhat)
    return - k0 * ( d - d0 ) * dhat

def compute_spring_forces(vertex1_array, vertex2_array, d_array, d0_array, k0_array):
    
    '''compute the net spring force at each vertex by iterating over the edges modeling springs.'''
    dist_array  = np.zeros_like(vertex1_array, dtype=float)
    force_array = np.zeros_like(vertex1_array, dtype=float)
    for i, (vertex1, vertex2, d, d0, k0) in enumerate(zip(vertex1_array, vertex2_array, d_array, d0_array, k0_array)):
        dist_array[i] = vertex1 - vertex2
        # compute the spring force given edge in the local scope
        f0   = spring_force( vertex1, vertex2, d, d0, k0 )

        # add that spring force in the appropriate entries of f0_array
        f0_array[meu[i,0]] += f0
        f0_array[meu[i,1]] -= f0
    return f0_array
        # TODO(optional, for plotting): compute the average of the vertex vectors

In [350]:
#precompute edges, which are the force generating simplices here
meu = mesh.edges_unique
vid1_array, vid2_array = np.array(meu[:,0],dtype=int), np.array( meu[:,1],dtype=int)
vertex1_array = np.array ( mesh.vertices[vid1_array]  , dtype=float) 
vertex2_array = np.array ( mesh.vertices[vid2_array]  , dtype=float) 

#initialize spring force parameters
d0_array = np.array(mesh.edges_unique_length, dtype=float)
k0_array = 1 + np.zeros_like(vertex1_array[...,0], dtype=float)
f0_array = np.array(np.zeros_like(mesh.vertices, dtype=float))

#compute the current spring deformation states for each edge
vertex1_array = np.array(mesh.vertices[vid1_array],dtype=float)
vertex2_array = np.array( mesh.vertices[vid2_array],dtype=float)
d_array = np.array(mesh.edges_unique_length, dtype=float)

# compute the net spring force for each vertex
f0_array = compute_spring_forces(vertex1_array, vertex2_array, d_array, d0_array, k0_array)

In [360]:
def test_spring_force_computation(mesh):
    #precompute edges, which are the force generating simplices here
    meu = mesh.edges_unique
    vid1_array, vid2_array = np.array(meu[:,0],dtype=int), np.array( meu[:,1],dtype=int)
    vertex1_array = np.array ( mesh.vertices[vid1_array]  , dtype=float) 
    vertex2_array = np.array ( mesh.vertices[vid2_array]  , dtype=float) 

    #initialize spring force parameters
    d0_array = np.array(mesh.edges_unique_length, dtype=float)
    k0_array = 1 + np.zeros_like(vertex1_array[...,0], dtype=float)
    f0_array = np.array(np.zeros_like(mesh.vertices, dtype=float))

    #compute the current spring deformation states for each edge
    vertex1_array = np.array(mesh.vertices[vid1_array],dtype=float)
    vertex2_array = np.array( mesh.vertices[vid2_array],dtype=float)
    d_array = np.array(mesh.edges_unique_length, dtype=float)

    assert ((mesh.edges_unique[mesh.edges_unique_inverse] == mesh.edges_sorted).all())

    def length(vertex1, vertex2):
        return np.linalg.norm( vertex1 - vertex2)

    #test mesh.edges_unique_length gives the correct edge length 
    assert ( length(vertex1_array[0],vertex2_array[0]) == mesh.edges_unique_length[0] )
    assert ( length(vertex1_array[1],vertex2_array[1]) == mesh.edges_unique_length[1] )
    assert ( length(vertex1_array[2],vertex2_array[2]) == mesh.edges_unique_length[2] )
    
    #test type consistency as a proxy test that the types are not tracked-array instances
    assert ( type(dist_array) == type (d0_array) )
    assert ( type(f0_array) == type (k0_array) )
    assert ( type(vertex1_array) == type (d0_array) )
    assert ( type(vertex2_array) == type (d0_array) )
    assert ( type(vid1_array) == type (d0_array) )
    assert ( type(vid2_array) == type (d0_array) )

    # test that spring_force pushes vertex1 in the direction of vertex2 when the spring is twice the equilibrium length, d0
    vertex1, vertex2, d, d0, k0  = (np.array([ 0.71367072, -0.45411208,  0.53182509]),np.array([ 0.54048159, -0.63460453,  0.61003465]),0.26208535311672354,0.26208535311672354,1.0)
    dhat = (vertex1 - vertex2)
    dhat/= np.linalg.norm(dhat)

    # test that an extended spring is attractive in the edge direction
    f0   = spring_force( vertex1, vertex2, d, d0/2, k0 )
    assert ( np.isclose ( f0.dot(dhat)/np.linalg.norm(f0) , 1 ) ) 
    # test that a compressed spring is repulsive in the edge direction
    f0   = spring_force( vertex1, vertex2, d, d0*2, k0 )
    assert ( np.isclose ( f0.dot(dhat)/np.linalg.norm(f0) , -1 ) ) 
    
    # compute the net spring force for each vertex
    f0_array = compute_spring_forces(vertex1_array, vertex2_array, d_array, d0_array, k0_array)
    assert (np.isclose( np.mean(f0_array) , 0 ) )  #test average spring force is zero
    assert ( np.isclose( np.var(f0_array) , 0 ) )  #test variance of spring force at least 0.02
    return True

In [361]:
test_spring_force_computation(mesh)

True

In [364]:
vertex_array = np.array(mesh.vertices, dtype=float)

# njit the spring forces (unjitted is ~100ms. njit'ing will let integrate_forward_n_steps become njitt'd
    - Don't spend too long on this!

In [9]:
import os, trimesh, numpy as np
from lib.spring import *
os.chdir(f'Data/spherical_meshes')
mesh = trimesh.load('spherical_mesh_64.stl')
os.chdir('../..')
#subtract the center of mass
mesh.vertices -= mesh.center_mass
#normalize the mean radius to 1
mesh.vertices /= np.cbrt(mesh.volume*3/(4*np.pi))

#precompute edges, which are the force generating simplices here
meu = mesh.edges_unique
vid1_array, vid2_array = np.array(meu[:,0],dtype=int), np.array( meu[:,1],dtype=int)
vertex1_array = np.array ( mesh.vertices[vid1_array]  , dtype=float) 
vertex2_array = np.array ( mesh.vertices[vid2_array]  , dtype=float) 
vertex_array = np.array(mesh.vertices, dtype=float) #only needed for shape

#initialize spring force parameters
d0_array = np.array(mesh.edges_unique_length, dtype=float)
k0_array = 1 + np.zeros_like(vertex1_array[...,0], dtype=float)
f0_array = np.array(np.zeros_like(mesh.vertices, dtype=float))

#compute the current spring deformation states for each edge
vertex1_array = np.array(mesh.vertices[vid1_array],dtype=float)
vertex2_array = np.array( mesh.vertices[vid2_array],dtype=float)
d_array = np.array(mesh.edges_unique_length, dtype=float)

# compute the net spring force for each vertex
f0_array = compute_spring_forces(vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array)

face_normals all zero, ignoring!


In [11]:
from numba import njit

In [15]:
#njit spring_force
#compute spring force for a given edge configuration
@njit
def spring_force( vertex1, vertex2, d, d0, k0 ):
    '''returns the spring_force for vertex1 directed towards vertex2.  '''
    dhat = (vertex2 - vertex1)
    dhat /= np.linalg.norm(dhat)
    return - k0 * ( d - d0 ) * dhat

In [19]:
# test that spring_force pushes vertex1 in the direction of vertex2 when the spring is twice the equilibrium length, d0
vertex1, vertex2, d, d0, k0  = (np.array([ 0.71367072, -0.45411208,  0.53182509]),np.array([ 0.54048159, -0.63460453,  0.61003465]),0.26208535311672354,0.26208535311672354,1.0)
dhat = (vertex1 - vertex2)
dhat/= np.linalg.norm(dhat)

# test that an extended spring is attractive in the edge direction
f0   = spring_force( vertex1, vertex2, d, d0/2, k0 )
assert ( np.isclose ( f0.dot(dhat)/np.linalg.norm(f0) , 1 ) ) 
# test that a compressed spring is repulsive in the edge direction
f0   = spring_force( vertex1, vertex2, d, d0*2, k0 )
assert ( np.isclose ( f0.dot(dhat)/np.linalg.norm(f0) , -1 ) ) 

In [83]:
@njit
def compute_spring_forces(vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array):
    '''compute the net spring force at each vertex by iterating over the edges modeling springs.'''
    f0_array = 0. * vertex_array
    # f0_array = np.array(np.zeros_like(vertex_array, dtype=float))
    # dist_array  = np.zeros_like(vertex1_array, dtype=float)
    # force_array = np.zeros_like(vertex1_array, dtype=float)
    #     for i, (vertex1, vertex2, d, d0, k0) in enumerate(zip(vertex1_array, vertex2_array, d_array, d0_array, k0_array)):
    # dist_array[i] = vertex1 - vertex2
    imax = vertex1_array.shape[0]
    for i in range(imax):
        vertex1, vertex2, d, d0, k0 = vertex1_array[i], vertex2_array[i], d_array[i], d0_array[i], k0_array[i]
    # compute the spring force given edge in the local scope
        f0   = spring_force( vertex1, vertex2, d, d0, k0 )

        # add that spring force in the appropriate entries of f0_array
        f0_array[vid1_array[i]] = f0_array[vid1_array[i]] + f0
        f0_array[vid2_array[i]] = f0_array[vid2_array[i]] - f0
    return f0_array

In [84]:
#TODO: test that none of these types are tracked
# compute_spring_forces(
# [
# vertex1_array.shape, 
# vertex2_array.shape, 
# d_array.shape, 
# d0_array.shape, 
# k0_array.shape,
# vid1_array.shape, 
# vid2_array.shape, 
# vertex_array.shape
# ]

[(756, 3), (756, 3), (756,), (756,), (756,), (756,), (756,), (254, 3)]

In [86]:
# #numba bug is in this line
# for i, (vertex1, vertex2, d, d0, k0) in enumerate(zip(vertex1_array, vertex2_array, d_array, d0_array, k0_array)):
#     pass
# #note that dimensions must agree because it works in core python
# #TODO: try rewriting ^this as
# imax = vertex1_array.shape[0]
# for i in range(imax):
#     vertex1, vertex2, d, d0, k0 = vertex1_array[i], vertex2_array[i], d_array[i], d0_array[i], k0_array[i]
#     pass

In [93]:
# njit compute_spring_forces
# compute_spring_forces(vertex1_array, vertex2_array, d_array, d0_array, k0_array, vid1_array, vid2_array, vertex_array)

In [ ]:
#TODO(later, in openoffice): make a nice plot of a some example forces on an example triangle.

# exploring scipy.interpolate

In [98]:
import scipy.interpolate as interpolate

In [103]:
# Smooth bivariate spline approximation.
# interpolate.SmoothBivariateSpline?

In [104]:
# Weighted least-squares bivariate spline approximation.
# interpolate.LSQBivariateSpline?

In [107]:
# Convenience function for polynomial interpolation.
# interpolate.barycentric_interpolate?

# The interpolating polynomial for a set of points
# interpolate.BarycentricInterpolator?

In [113]:
# Piecewise cubic, C1 smooth, curvature-minimizing interpolant in 2D.
scipy.interpolate.CloughTocher2DInterpolator?

# represent a triangular mesh (sphere) as a bivariate B-spline smooth surface.

In [ ]:
#Do I want a scale-invariant renormalization group formulated by the continuous group (B-spline bivariate model) or by the barycentric group?
# I want the B-spline bivariate model for mechanics and for the electrophysiology.



In [111]:
# TODO: Find a bivariate B-spline representation of a surface.
# interpolate.bisplrep?

In [109]:
# TODO: Evaluate a bivariate B-spline and its derivatives.
# interpolate.bisplev?